threading 模块提供了一种在单个进程内部并发地运行多个 线程 (从进程分出的更小单位) 的方式。 它允许创建和管理线程，以便能够平行地执行多个任务，并共享内存空间。

线程特别适用于 I/O 密集型的任务，如文件操作或发送网络请求，在此类任务中大部分时间都会消耗于等待外部资源。


重点：threading 模块是在单个进程内部操作的，这意味着所有线程共享相同的内存空间。

In [4]:
import threading
import time
import random
links = [
    "https://www.baidu.com",
    "https://www.google.com",
    "https://www.youtube.com",
    "https://www.yahoo.com",
    "https://www.bing.com",
    "https://www.taobao.com",
    "https://www.jd.com",
    "https://www.amazon.com",
    "https://www.twitter.com",
    "https://www.facebook.com",
]

def fetch(url:str,delay:float=0.1):
    print("开始抓取：",url)
    time.sleep(delay)
    print("抓取完毕：",url)

task_threads = []
for link in links:
    t = threading.Thread(target=fetch,args=(link,),kwargs={"delay":random.randint(5,20)/10})
    task_threads.append(t)
# 保证任务同时启动
for t in task_threads:
    t.start()
    # t.join()

for t in task_threads:
    t.join()

print("所有任务完成")


开始抓取：开始抓取： https://www.google.com
 https://www.baidu.com
开始抓取： https://www.youtube.com
开始抓取： https://www.yahoo.com
开始抓取： https://www.bing.com
开始抓取： https://www.taobao.com
开始抓取： https://www.jd.com
开始抓取： https://www.amazon.com
开始抓取： https://www.twitter.com
开始抓取： https://www.facebook.com
抓取完毕： https://www.amazon.com
抓取完毕： https://www.jd.com
抓取完毕： https://www.taobao.com
抓取完毕：抓取完毕： https://www.twitter.com
 https://www.yahoo.com
抓取完毕： https://www.facebook.com
抓取完毕： https://www.bing.com
抓取完毕： https://www.youtube.com
抓取完毕： https://www.baidu.com
抓取完毕： https://www.google.com
所有任务完成


## 线程局部数据

线程局部数据是指具有线程专属值的数据。 如果你希望某些数据是线程局部数据，则创建一个 local 对象并使用其属性:

它是如何实现的？（进阶理解）
threading.local对象内部维护了一个大字典。这个字典的 key 是每个线程的唯一标识符（id），value 是该线程专属的一个小字典，用来存储这个线程设置的属性（比如 number=11）


当你在一个线程中访问 mydata.number时，Python 解释器会暗中执行以下操作：

获取当前线程的 ID。

在 mydata内部的大字典里，找到对应这个线程 ID 的小字典。

从那个小字典里读取或设置 number属性。

正是通过这种“一个线程对应一个私有字典”的机制，实现了数据的自动隔离。为了保证在操作这个大字典时的线程安全，其内部也使用了锁

In [5]:
import threading
mydata = threading.local()
mydata.x = 1
mydata.y = 2

print(mydata.__dict__)



{'x': 1, 'y': 2}


In [6]:
import threading

# 创建一个线程局部存储对象。这是整个程序中的“全局”对象。
mydata = threading.local()
# 初始化一个日志列表，用于记录每个线程的操作顺序和结果
log = []

def f():
    # 在线程函数开始时，为当前线程的mydata设置一个独有的属性
    mydata.number = 0
    # 获取mydata的所有属性及其值，并排序，然后记录到log中
    items = sorted(mydata.__dict__.items())
    log.append(items)  # 此时记录的是 [('number', 0)]
    
    # 修改当前线程的mydata的number属性
    mydata.number = 11
    # 再次记录修改后的值
    log.append(mydata.number)  # 记录 11

# 在主线程中也为mydata设置一个属性，注意，这和子线程中的mydata是隔离的！
mydata.main_thread_data = "I'm from main thread"

# 创建并启动子线程
thread = threading.Thread(target=f)
thread.start()

# 等待子线程执行完毕
thread.join()

# 打印最终的日志内容
print("Log from the thread:", log)
# 打印主线程中mydata的number属性（子线程的修改不影响这里）
print("mydata.number in main thread:", getattr(mydata, 'number', 'Attribute not found'))


Log from the thread: [[('number', 0)], 11]
mydata.number in main thread: Attribute not found


In [ ]:
import threading

# 使用一个普通的全局变量
real_global_data = {'number': 0}
log_global = []

def f_conflict():
    real_global_data['number'] = 0  # 重置（但多个线程会互相覆盖）
    items = sorted(real_global_data.items())
    log_global.append(items)
    
    real_global_data['number'] = 11  # 修改会影响所有线程！
    log_global.append(real_global_data['number'])

threads = []
for i in range(3):  # 启动3个线程来制造混乱
    t = threading.Thread(target=f_conflict)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("Log with global variable (can be messy):", log_global)


Log with global variable (can be messy): [[('number', 0)], 11, [('number', 0)], 11, [('number', 0)], 11]
